# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7ff6c0b94280>
├── 'a' --> tensor([[ 0.1796,  0.3682, -0.1221],
│                   [-0.4592, -0.5511, -0.4903]])
└── 'x' --> <FastTreeValue 0x7ff6c0b94370>
    └── 'c' --> tensor([[-1.8354, -0.0019, -0.2760, -1.4201],
                        [-1.0434, -1.7258,  0.1108, -0.0780],
                        [-0.3169,  1.0179,  0.3503,  0.7881]])

In [4]:
t.a

tensor([[ 0.1796,  0.3682, -0.1221],
        [-0.4592, -0.5511, -0.4903]])

In [5]:
%timeit t.a

60.7 ns ± 0.0924 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7ff6c0b94280>
├── 'a' --> tensor([[-0.7015,  0.7213, -1.0666],
│                   [-2.2994, -0.1490,  0.6977]])
└── 'x' --> <FastTreeValue 0x7ff6c0b94370>
    └── 'c' --> tensor([[-1.8354, -0.0019, -0.2760, -1.4201],
                        [-1.0434, -1.7258,  0.1108, -0.0780],
                        [-0.3169,  1.0179,  0.3503,  0.7881]])

In [7]:
%timeit t.a = new_value

63.5 ns ± 0.0512 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[ 0.1796,  0.3682, -0.1221],
               [-0.4592, -0.5511, -0.4903]]),
    x: Batch(
           c: tensor([[-1.8354, -0.0019, -0.2760, -1.4201],
                      [-1.0434, -1.7258,  0.1108, -0.0780],
                      [-0.3169,  1.0179,  0.3503,  0.7881]]),
       ),
)

In [10]:
b.a

tensor([[ 0.1796,  0.3682, -0.1221],
        [-0.4592, -0.5511, -0.4903]])

In [11]:
%timeit b.a

60.2 ns ± 0.0172 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[ 1.9789, -0.9663,  0.0320],
               [ 0.3517, -0.3929,  0.8135]]),
    x: Batch(
           c: tensor([[-1.8354, -0.0019, -0.2760, -1.4201],
                      [-1.0434, -1.7258,  0.1108, -0.0780],
                      [-0.3169,  1.0179,  0.3503,  0.7881]]),
       ),
)

In [13]:
%timeit b.a = new_value

479 ns ± 0.248 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

821 ns ± 0.274 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

11.4 µs ± 61.2 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

136 µs ± 274 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

139 µs ± 263 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7ff609e33610>
├── 'a' --> tensor([[[ 0.1796,  0.3682, -0.1221],
│                    [-0.4592, -0.5511, -0.4903]],
│           
│                   [[ 0.1796,  0.3682, -0.1221],
│                    [-0.4592, -0.5511, -0.4903]],
│           
│                   [[ 0.1796,  0.3682, -0.1221],
│                    [-0.4592, -0.5511, -0.4903]],
│           
│                   [[ 0.1796,  0.3682, -0.1221],
│                    [-0.4592, -0.5511, -0.4903]],
│           
│                   [[ 0.1796,  0.3682, -0.1221],
│                    [-0.4592, -0.5511, -0.4903]],
│           
│                   [[ 0.1796,  0.3682, -0.1221],
│                    [-0.4592, -0.5511, -0.4903]],
│           
│                   [[ 0.1796,  0.3682, -0.1221],
│                    [-0.4592, -0.5511, -0.4903]],
│           
│                   [[ 0.1796,  0.3682, -0.1221],
│                    [-0.4592, -0.5511, -0.4903]]])
└── 'x' --> <FastTreeValue 0x7ff609e33a00>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

32.5 µs ± 80.2 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7ff609e33b80>
├── 'a' --> tensor([[ 0.1796,  0.3682, -0.1221],
│                   [-0.4592, -0.5511, -0.4903],
│                   [ 0.1796,  0.3682, -0.1221],
│                   [-0.4592, -0.5511, -0.4903],
│                   [ 0.1796,  0.3682, -0.1221],
│                   [-0.4592, -0.5511, -0.4903],
│                   [ 0.1796,  0.3682, -0.1221],
│                   [-0.4592, -0.5511, -0.4903],
│                   [ 0.1796,  0.3682, -0.1221],
│                   [-0.4592, -0.5511, -0.4903],
│                   [ 0.1796,  0.3682, -0.1221],
│                   [-0.4592, -0.5511, -0.4903],
│                   [ 0.1796,  0.3682, -0.1221],
│                   [-0.4592, -0.5511, -0.4903],
│                   [ 0.1796,  0.3682, -0.1221],
│                   [-0.4592, -0.5511, -0.4903]])
└── 'x' --> <FastTreeValue 0x7ff6c0ba1700>
    └── 'c' --> tensor([[-1.8354, -0.0019, -0.2760, -1.4201],
                        [-1.0434, -1.7258,  0.1108, -0.0780],
                 

In [23]:
%timeit t_cat(trees)

30.8 µs ± 86.9 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

58.9 µs ± 149 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    a: tensor([[[ 0.1796,  0.3682, -0.1221],
                [-0.4592, -0.5511, -0.4903]],
       
               [[ 0.1796,  0.3682, -0.1221],
                [-0.4592, -0.5511, -0.4903]],
       
               [[ 0.1796,  0.3682, -0.1221],
                [-0.4592, -0.5511, -0.4903]],
       
               [[ 0.1796,  0.3682, -0.1221],
                [-0.4592, -0.5511, -0.4903]],
       
               [[ 0.1796,  0.3682, -0.1221],
                [-0.4592, -0.5511, -0.4903]],
       
               [[ 0.1796,  0.3682, -0.1221],
                [-0.4592, -0.5511, -0.4903]],
       
               [[ 0.1796,  0.3682, -0.1221],
                [-0.4592, -0.5511, -0.4903]],
       
               [[ 0.1796,  0.3682, -0.1221],
                [-0.4592, -0.5511, -0.4903]]]),
    x: Batch(
           c: tensor([[[-1.8354, -0.0019, -0.2760, -1.4201],
                       [-1.0434, -1.7258,  0.1108, -0.0780],
                       [-0.3169,  1.0179,  0.3503,  0.7881]],
         

In [26]:
%timeit Batch.stack(batches)

78.7 µs ± 199 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    a: tensor([[ 0.1796,  0.3682, -0.1221],
               [-0.4592, -0.5511, -0.4903],
               [ 0.1796,  0.3682, -0.1221],
               [-0.4592, -0.5511, -0.4903],
               [ 0.1796,  0.3682, -0.1221],
               [-0.4592, -0.5511, -0.4903],
               [ 0.1796,  0.3682, -0.1221],
               [-0.4592, -0.5511, -0.4903],
               [ 0.1796,  0.3682, -0.1221],
               [-0.4592, -0.5511, -0.4903],
               [ 0.1796,  0.3682, -0.1221],
               [-0.4592, -0.5511, -0.4903],
               [ 0.1796,  0.3682, -0.1221],
               [-0.4592, -0.5511, -0.4903],
               [ 0.1796,  0.3682, -0.1221],
               [-0.4592, -0.5511, -0.4903]]),
    x: Batch(
           c: tensor([[-1.8354, -0.0019, -0.2760, -1.4201],
                      [-1.0434, -1.7258,  0.1108, -0.0780],
                      [-0.3169,  1.0179,  0.3503,  0.7881],
                      [-1.8354, -0.0019, -0.2760, -1.4201],
                      [-1.0434, -

In [28]:
%timeit Batch.cat(batches)

143 µs ± 376 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

310 µs ± 3.6 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
